<a href="https://colab.research.google.com/github/KarlaLutz/Trabalho_Cripto_Turma9_BigData_FGV/blob/main/C%C3%B3digo_Decis%C3%A3o_de_Compra_Venda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importação de Pacotes**

In [3]:
import pickle
import time
from simple_robot import feature_eng, api_get, api_post, get_result, compute_quantity, how_much_i_have
import numpy as np
import pandas as pd
import requests
import json

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# **Carregando Modelo Criado**

In [4]:
from google.colab import drive
drive.mount('/drive')

model = pickle.load(open('/drive/MyDrive/Colab Notebooks/Trabalho Cripto/model_mls.pickle', 'rb'))
token = '64559d394d62025'#'token_dummy_001'
ticker = 'BTCUSDT'
valor_compra_venda = 10

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


# **Adicionando as novas features**

In [5]:
# Função para converter a hora da transação em hora cheia 
def add_hora_cheia (row):
   import logging
   from datetime import datetime
   logger = logging.getLogger(__name__)
   try:
      retorno = pd.Timestamp(year = row.datetime.year,  
                            month = row.datetime.month, 
                            day = row.datetime.day, 
                            hour = row.datetime.hour, 
                            second = 0)    
   except:
     
      retorno = pd.Timestamp(year = datetime.today().year,  
                            month = datetime.today().month, 
                            day = datetime.today().day, 
                            hour = datetime.today().hour, 
                            second = 0) 
   return retorno

In [6]:
def add_features_df(df):

    #Adicionando a hora cheia ao dataframe
    #print(type(df))
    df['hora_cheia'] = df.apply (lambda row: add_hora_cheia(row), axis=1)

    #Buscando a ultima hora e convertendo em um timestamp (string) para uso na chamada da API das features adicionais
    df_ts = df.iloc[[np.argmax(df['datetime'])]]
    df_ts['hora_cheia'] = df_ts.apply (lambda row: add_hora_cheia(row), axis=1)
    timestamp = (pd.to_datetime(df_ts.hora_cheia, unit='s').astype(np.int64) // 10 ** 9)
    print(timestamp)
    timestamp = str(timestamp)[7:17]

    #Chamando a API das features com o ultimo timestamp
    url = "https://api.lunarcrush.com/v2?data=assets&key=qc53fwv86ympdmcwqzb7ml&symbol=BTC&data_points=1&interval=hour&start="+timestamp+"&end="+timestamp
    request = requests.get(url)
    retorno = request.content 
    data = json.loads(retorno)

    #Tratando as variaveis do dataframe com as features
    json_object = json.dumps(data['data'][0]['timeSeries'][0], indent = 4) 
    from pandas.io.json import json_normalize
    df_add = json_normalize(data['data'][0]['timeSeries'])
    df_add['hora_cheia'] = df_add.apply (lambda row: pd.to_datetime(row.time, unit='s'), axis=1)
    df_add = df_add[['hora_cheia','url_shares','reddit_posts','reddit_posts_score','reddit_comments_score','tweet_quotes','tweet_retweets','tweet_replies','tweet_sentiment_impact3','average_sentiment','news','price_score','social_impact_score','correlation_rank','volatility','alt_rank','market_cap_rank','percent_change_24h_rank','volume_24h_rank','social_volume_24h_rank','social_dominance']]
    
    #Unindo o dataframe original com o novo com as features
    df = pd.merge(df, df_add, how='left',  on = 'hora_cheia')
    df.fillna(0, inplace=True)
    return df


# **Verificação de Valores e Decisão de Compra/Venda**

In [7]:
import logging
logger = logging.getLogger(__name__)

while True:
    
    try:

        # Pegando o OHLC dos últimos 500 minutos
        df = api_post('cripto_quotation', {'token': token, 'ticker': ticker})

        while isinstance(df, str):
          df = api_post('cripto_quotation', {'token': token, 'ticker': ticker})

        # Adicionando as features da API
        df = add_features_df(df)

        # Realizando a engenharia de features
        df = feature_eng(df)
        
        # Isolando a linha mais recente
        df_last = df.iloc[[np.argmax(df['time'])]]
        
        # Calculando tendência, baseada no modelo linear criado
        tendencia = model.predict(df_last).iloc[0]
        
        # A quantidade de cripto que será comprada/ vendida depende do valor_compra_venda e da cotação atual
        qtdade = compute_quantity(coin_value = df_last['close'], invest_value = valor_compra_venda, significant_digits = 2)
        
        # Print do datetime atual
        print('-------------------')
        now = pd.to_datetime('now')
        print(f'{now}')
        
        if tendencia > 0.02:
            # Modelo detectou uma tendência positiva
            print(f"Tendência positiva de {str(tendencia)}")
            
            # Verifica quanto dinheiro tem em caixa
            qtdade_money = how_much_i_have('money', token)
            
            if qtdade_money>0:
                # Se tem dinheiro, tenta comprar o equivalente a qtdade ou o máximo que o dinheiro permitir
                max_qtdade = compute_quantity(coin_value = df_last['close'], invest_value = qtdade_money, significant_digits = 2)
                qtdade = min(qtdade, max_qtdade)
                
                # Realizando a compra
                print(f'Comprando {str(qtdade)} {ticker}')
                api_post('buy', payload = {'token': token, 'ticker': ticker, 'quantity': qtdade})
            
        elif tendencia < -0.02:
            # Modelo detectou uma tendência negativa
            print(f"Tendência negativa de {str(tendencia)}")
            
            # Verifica quanto tem da moeda em caixa
            qtdade_coin = how_much_i_have(ticker, token)
            
            if qtdade_coin>0:
                # Se tenho a moeda, vou vender!
                qtdade = min(qtdade_coin, qtdade)
                print(f'Vendendo {str(qtdade)} {ticker}')
                api_post('sell', payload = {'token': token, 'ticker': ticker, 'quantity':qtdade})
        else:
            # Não faz nenhuma ação, espera próximo loop
            print(f"Tendência neutra de {str(tendencia)}. Nenhuma ação realizada")
        
        # Print do status após cada iteração
        print(api_post('status', payload = {'token': token}))
        time.sleep(60)
    except KeyError as e:
        logger.exception(e) 
        continue

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


499    1634068800
Name: hora_cheia, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


-------------------
2021-10-12 20:59:03.752521
Tendência negativa de -0.08215376103839324
     ticker  quantity
0  DOGEUSDT      0.00
1   BTCUSDT      0.00
2     money   9999.95
3     total   9999.95


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


499    1634072400
Name: hora_cheia, dtype: int64


IndexError: ignored